In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

# <font color = "blue">Self-Assessment: Cheaper Schedule?</font>

In [ ]:
# Solve the worker scheduling problem
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

workers = ['KC', 'DH', 'HB', 'SC', 'KS', 'NK']
min_hours = [8, 8, 8, 8, 7, 7]
hourly_rate = [25, 26, 24, 23, 28, 30]
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
hours_to_staff = [14, 14, 14, 14, 14]

min_hours_dict = dict(zip(workers, min_hours))  # zip two lists into dict
hourly_rate_dict = dict(zip(workers, hourly_rate))
hours_to_staff_dict = dict(zip(days, hours_to_staff))

hours_avail = 8

# instantiate Concrete Model
model = ConcreteModel()

# define variables
model.hrs = Var(workers, days, domain=NonNegativeReals)

# define objective function
model.total_cost = Objective(expr=sum(hourly_rate_dict[w] * model.hrs[w, d]
                                      for w in workers for d in days),
                             sense=minimize)

# define constraints
model.supply_ct = ConstraintList()
for w in workers:
    model.supply_ct.add(
        sum(model.hrs[w, d] for d in days) >= min_hours_dict[w])

model.demand_ct = ConstraintList()
for d in days:
    model.demand_ct.add(
        sum(model.hrs[w, d] for w in workers) == hours_to_staff_dict[d])

model.avail_ct = ConstraintList()
for w in workers:
    for d in days:
        model.avail_ct.add(model.hrs[w, d] <= hours_avail)

# solve
solver = SolverFactory('glpk')
solver.solve(model)

# convert model.hrs into a Pandas data frame for nicer display
schedule = pd.DataFrame([[model.hrs[w, d].value for d in days]
                         for w in workers],
                        index=workers,
                        columns=days)

# display
import babel.numbers as numbers  # needed to display as currency
print("The minimum total weekly cost = ",
      numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))
print("\nThe weekly hours assigned to each worker:")
for w in workers:
    print("Worker {0:s} is assigned {1:d} hours".format(w, int(sum(schedule.loc[w,:]))))
print("The number of hours to schedule for each worker is: ")
schedule

# <font color = "blue">Self-Assessment: Unbalanced Transportation Problem</font>

In [8]:
# problem data
mills = ['Bahamas', 'Hong Kong', 'Korea', 'Nigeria', 'Venezuela', 'Dummy']

max_supply = dict(zip(mills, [1000, 2000, 1000, 2000, 1000]))

dist_ctrs = [
    'Los Angeles', 'Chicago', 'London', 'Mexico City', 'Manila', 'Rome',
    'Tokyo', 'New York'
]

demand = dict(zip(dist_ctrs, [500, 800, 900, 900, 800, 100, 200, 700]))

ship_cost = [[2, 2, 3, 3, 7, 4, 7, 1], [6, 7, 8, 10, 2, 9, 4, 8],
             [5, 6, 8, 11, 4, 9, 1, 7], [14, 12, 6, 9, 11, 7, 5, 10],
             [4, 3, 5, 1, 9, 6, 11, 4], [0,0,0,0,0,0,0,0]]

ship_cost_dict = {
    mills[m]: dict(zip(dist_ctrs, ship_cost[m][:]))
    for m in range(len(mills))
}

# since cotton supply is 7000 and demand is 41900 we'll add a dummy
# distribution center with demand 200 for balance

dist_ctrs += ['Dummy']
for m in mills:
    ship_cost_dict[m]['Dummy'] = 0

import pandas as pd
pd.DataFrame(ship_cost_dict).transpose()

,Chicago,Dummy,London,Los Angeles,Manila,Mexico City,New York,Rome,Tokyo
Bahamas,2,0,3,2,7,3,1,4,7
Hong Kong,7,0,8,6,2,10,8,9,4
Korea,6,0,8,5,4,11,7,9,1
Nigeria,12,0,6,14,11,9,10,7,5
Venezuela,3,0,5,4,9,1,4,6,11
Dummy,0,0,0,0,0,0,0,0,0


In [2]:
ship_cost_dict

{'Bahamas': {'Los Angeles': 2,
  'Chicago': 2,
  'London': 3,
  'Mexico City': 3,
  'Manila': 7,
  'Rome': 4,
  'Tokyo': 7,
  'New York': 1,
  'Dummy': 0},
 'Hong Kong': {'Los Angeles': 6,
  'Chicago': 7,
  'London': 8,
  'Mexico City': 10,
  'Manila': 2,
  'Rome': 9,
  'Tokyo': 4,
  'New York': 8,
  'Dummy': 0},
 'Korea': {'Los Angeles': 5,
  'Chicago': 6,
  'London': 8,
  'Mexico City': 11,
  'Manila': 4,
  'Rome': 9,
  'Tokyo': 1,
  'New York': 7,
  'Dummy': 0},
 'Nigeria': {'Los Angeles': 14,
  'Chicago': 12,
  'London': 6,
  'Mexico City': 9,
  'Manila': 11,
  'Rome': 7,
  'Tokyo': 5,
  'New York': 10,
  'Dummy': 0},
 'Venezuela': {'Los Angeles': 4,
  'Chicago': 3,
  'London': 5,
  'Mexico City': 1,
  'Manila': 9,
  'Rome': 6,
  'Tokyo': 11,
  'New York': 4,
  'Dummy': 0}}

In [128]:
# problem data
mills = ['Bahamas', 'Hong Kong', 'Korea', 'Nigeria', 'Venezuela']

max_supply = dict(zip(mills, [1000, 2000, 1000, 2000, 1000]))

dist_ctrs = [
    'Los Angeles', 'Chicago', 'London', 'Mexico City', 'Manila', 'Rome',
    'Tokyo', 'New York'
]

demand = dict(zip(dist_ctrs, [500, 800, 900, 900, 800, 100, 200, 700]))

ship_cost = [[2, 2, 3, 3, 7, 4, 7, 1], [6, 7, 8, 10, 2, 9, 4, 8],
             [5, 6, 8, 11, 4, 9, 1, 7], [14, 12, 6, 9, 11, 7, 5, 10],
             [4, 3, 5, 1, 9, 6, 11, 4]]

ship_cost_dict = {
    mills[m]: dict(zip(dist_ctrs, ship_cost[m][:]))
    for m in range(len(mills))
}

# since cotton supply is 7000 and demand is 41900 we'll add a dummy
# distribution center with demand 200 for balance

dist_ctrs += ['Dummy']
for m in mills:
    ship_cost_dict[m]['Dummy'] = 0

demand['Dummy'] = sum(max_supply.values()) - sum(demand.values())

### Define Pyomo Model ###

from pyomo.environ import *

model = ConcreteModel()

# define variables
model.bolts = Var(mills, dist_ctrs, domain=NonNegativeReals)

# define objective function
model.total_cost = Objective(expr=sum(ship_cost_dict[m][d] *
                                      model.bolts[m, d] for m in mills
                                      for d in dist_ctrs),
                             sense=minimize)

# define constraints

model.supply_ct = ConstraintList()
for m in mills:
    model.supply_ct.add( sum( model.bolts[m,d] for d in dist_ctrs )
                        == max_supply[m] )

model.demand_ct = ConstraintList()
for d in dist_ctrs:
    model.demand_ct.add( sum( model.bolts[m,d] for m in mills ) == demand[d] ) 

# solve
solver = SolverFactory('glpk')
solver.solve(model)

# display

import babel.numbers as numbers  # needed to display as currency
print("Total Shipping Cost = ",
      numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))

import pandas as pd
bolts = pd.DataFrame([[model.bolts[m, d].value for d in dist_ctrs] for m in mills],
                         columns=dist_ctrs,
                         index=mills)
bolts

Total Shipping Cost =  $15,400.00


,Los Angeles,Chicago,London,Mexico City,Manila,Rome,Tokyo,New York,Dummy
Bahamas,0.0,300.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0
Hong Kong,0.0,100.0,0.0,0.0,800.0,0.0,0.0,0.0,1100.0
Korea,500.0,300.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0
Nigeria,0.0,0.0,900.0,0.0,0.0,100.0,0.0,0.0,1000.0
Venezuela,0.0,100.0,0.0,900.0,0.0,0.0,0.0,0.0,0.0


# <font color="blue">Self-Assessment:  Remove unneeded variables</font>

Revisit the worker scheduling problem from the textbook (3.4-15).  Start with the code above and modify it using Technique 3 to eliminate unneeded decision variables.  Your new solution should have 18 decision variables instead of 30 and the overall answer should be the same.

In [7]:
from pyomo.environ import *

model = ConcreteModel()

workers = ['KC','DH','HB','SC','KS','NK']
min_hr = [8,8,8,8,7,7]
hourly_rate = [25,26,24,23,28,30]
days = ['Mon','Tue','Wed','Thu','Fri']
hours_to_staff = [14,14,14,14,14]

min_hours = dict( zip( workers, min_hr ) )
rate_per_hour = dict( zip( workers, hourly_rate) )
tot_staff_hrs = dict( zip( days, hours_to_staff ) )

worker_day_avail={ ('KC','Mon'):6,('KC','Wed'):6,('KC','Fri'):6,
                   ('DH','Tue'):6,('DH','Thu'):6,
                   ('HB','Mon'):4,('HB','Tue'):8,('HB','Wed'):4,('HB','Fri'):4,
                   ('SC','Mon'):5,('SC','Tue'):5,('SC','Wed'):5,('SC','Fri'):5,
                   ('KS','Mon'):3,('KS','Wed'):3,('KS','Thu'):8,
                   ('NK','Thu'):6,('NK','Fri'):2 }

worker_days = worker_day_avail.keys() # makes a set of all the keys in the dictionary

# define variables
model.hrs = Var(worker_days, domain = NonNegativeReals) # one variable for each feasible worker-day pair

# define objective function
model.total_cost = Objective( expr = sum(rate_per_hour[w]*model.hrs[w,d] 
                                         for (w,d) in worker_days ), 
                            sense = minimize )

# define constraints 

model.supply_ct = ConstraintList()
for w in workers:
    model.supply_ct.add( sum( model.hrs[w,d] for d in days if (w,d) in worker_days ) >= min_hours[w])

model.demand_ct = ConstraintList()
for d in days:
    model.demand_ct.add( sum( model.hrs[w,d] for w in workers if (w,d) in worker_days) == tot_staff_hrs[d] )

model.avail_ct = ConstraintList()
for (w,d) in worker_days:
    model.avail_ct.add(model.hrs[w,d] <= worker_day_avail[w,d])
    
# solve
solver = SolverFactory('glpk')
solver.solve(model)

# convert model.hrs into a Pandas data frame for nicer display
import pandas as pd  
schedule = pd.DataFrame( 0, index = workers, columns = days)
for (w,d) in worker_days:
    schedule.loc[w,d] = model.hrs[w,d].value

# display
import babel.numbers as numbers  # needed to display as currency
print("The minimum total weekly cost = ",
      numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))
print("The number of hours to schedule for each worker is: ")
schedule

The minimum total weekly cost =  $1,755.00
The number of hours to schedule for each worker is: 


,Mon,Tue,Wed,Thu,Fri
KC,2.0,0.0,3.0,0.0,4.0
DH,0.0,2.0,0.0,6.0,0.0
HB,4.0,7.0,4.0,0.0,4.0
SC,5.0,5.0,5.0,0.0,5.0
KS,3.0,0.0,2.0,2.0,0.0
NK,0.0,0.0,0.0,6.0,1.0


# <font color="blue">Self-Assessment: Multiple Products</font>

In [1]:
import pyomo

# using openpyxl
from openpyxl import load_workbook
wb = load_workbook(filename='data/transp_prob_multi.xlsx', data_only=True)
sheet = wb.active

# specify upper left and lower right cells, returns a list or list of lists representing rows
# for a single value read_range(sheet,'A11')
# for a list of values in a column or row read_range(sheet,'A11','N11')
# for nested lists of values (array-like) read_range(sheet,'A11','D23')
def read_range(sheet, begin, *argv):
    if len(argv)>0:
        end = argv[0]
        table = sheet[begin:end]
        height = len(table)
        width = len(table[0])
        if height == 1 or width == 1:
            # for a single row or column produce a list
            tmp = [cell.value for row in table for cell in row]
        else:
            # for an array of cells produces a list of row lists
            tmp = [[cell.value for cell in row] for row in table]
    else:
        tmp = sheet[begin].value
    return (tmp)

warehouses = read_range(sheet, 'A3', 'A5')
stores = read_range(sheet, 'B3', 'B7')
products = read_range(sheet, 'C3', 'C4')
routes = {(p, w, s) for [p, w, s] in read_range(sheet, 'E3', 'G20')}
wares_stores = {(w,s) for (p,w,s) in routes}
capacity = read_range(sheet, 'A11')
cost = {(p, w, s): c for [p, w, s, c] in read_range(sheet, 'E3', 'H20')}
supply = {(p, w): q for [p, w, q] in read_range(sheet, 'J3', 'L8')}
demand = {(p, s): q for [p, s, q] in read_range(sheet, 'N3', 'P12')}

# throw an error if total supply and demand do not match
for p in products:
    assert (sum(supply[p, w] for w in warehouses
                if (p, w) in supply.keys()) == sum(demand[p, s]
                                                   for s in stores
                                                   if (p, s) in demand.keys()))

from pyomo.environ import *

model = ConcreteModel()

model.transp = Var(routes, domain=NonNegativeReals)

model.total_cost = Objective(expr=sum(cost[p,w,s] * model.transp[p,w,s] for (p,w,s) in routes) )

model.supply_ct = ConstraintList()
for p in products:
    for w in warehouses:
        model.supply_ct.add( sum( model.transp[p,w,s] for s in stores if (p,w,s) in routes) == supply[p,w] )


model.demand_ct = ConstraintList()
for p in products:
    for s in stores:
        model.demand_ct.add( sum( model.transp[p,w,s] for w in warehouses if (p,w,s) in routes) == demand[p,s] )

model.capacity_ct = ConstraintList()
for (w,s) in wares_stores:
    model.capacity_ct.add( sum(model.transp[p, w, s] for p in products if (p,w,s) in routes)  <= capacity)

# solve and display
solver = SolverFactory('glpk')
solver.solve(model)

# convert model.hrs into a Pandas data frame for nicer display
import pandas as pd
transp_pA = pd.DataFrame(0, index=warehouses, columns=stores)
transp_pB = pd.DataFrame(0, index=warehouses, columns=stores)
for (w, s) in wares_stores:
    transp_pA.loc[w, s] = model.transp['pA', w, s].value
    transp_pB.loc[w, s] = model.transp['pB', w, s].value

# display
import babel.numbers as numbers  # needed to display as currency
print("The minimum total transportation cost = ",
      numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))

from IPython.display import display
print("\nThe transported amounts of product A: ")
display(transp_pA)
print("\nThe transported amounts of product B: ")
display(transp_pB)

The minimum total transportation cost =  $23,700.00

The transported amounts of product A: 


,sA,sB,sC,sD,sE
wA,200.0,0.0,200.0,0.0,0.0
wB,0.0,100.0,0.0,0.0,0.0
wC,0.0,0.0,100.0,100.0,300.0



The transported amounts of product B: 


,sA,sB,sC,sD,sE
wA,200.0,100.0,200.0,0.0,0.0
wB,0.0,0.0,0.0,100.0,0.0
wC,0.0,0.0,0.0,0.0,400.0
